In [1]:
import pandas as pd
# Load Binance BNB/USDT 1-minute data
df = pd.read_csv('/Users/mchildress/Active Code/ts_basics/data/bnbusdt_1m.csv',
                 parse_dates=True, index_col=0)
# Use df for downstream processing
data = df  # rename as needed by notebook

In [ ]:
# 2. Preprocess & split
from sklearn.preprocessing import StandardScaler
# use 'close' price
scaler = StandardScaler()
series = scaler.fit_transform(df[['close']].values)
# parameters
lookback, horizon = 720, 96
split = int(len(series) * 0.8)
train_series, val_series = series[:split], series[split:]

# 3. Dataset & DataLoader
import torch
from torch.utils.data import Dataset, DataLoader

class TSData(Dataset):
    def __init__(self, arr, lookback, horizon):
        X, Y = [], []
        for i in range(len(arr) - lookback - horizon + 1):
            X.append(arr[i:i+lookback])
            Y.append(arr[i+lookback:i+lookback+horizon])
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
        self.Y = torch.tensor(Y, dtype=torch.float32).unsqueeze(-1)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

train_ds = TSData(train_series, lookback, horizon)
val_ds   = TSData(val_series,   lookback, horizon)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False)

/var/folders/yj/3s0hc5nn3qlg4lqp7wmfgq_c0000gn/T/ipykernel_94613/2140110104.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)


In [ ]:
# 4. Import PRformer
import sys, pathlib
repo_root = pathlib.Path("/Users/mchildress/Active Code/ts_basics/Modeling/Transformer")
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from prformer import PRformer
import torch.nn as nn

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
# 5. Instantiate PRformer, optimizer & loss
model = PRformer(
    input_len=lookback,
    output_len=horizon,
    d_model=64,
    n_heads=4,
    num_layers=3,
    dropout=0.1
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

In [ ]:
# 6. Training loop
epochs = 10
for ep in range(epochs):
    model.train()
    total_loss = 0.0
    for X, Y in train_loader:
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        preds = model(X)  # PRformer takes (B, L, 1) and returns (B, Horizon, 1)
        loss = criterion(preds, Y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {ep+1}/{epochs} — train MSE: {total_loss/len(train_loader):.5f}")